<a href="https://colab.research.google.com/github/NorahAbdulaziz/VideoCaptionModel/blob/main/newTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from PIL import Image
import PIL 
import os
import math
from skimage.metrics import structural_similarity as compare_ssim
import argparse
import imutils
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import shutil
import tqdm
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import config


In [ ]:
#copied cell
#https://morioh.com/p/0cbc8ac635b9


def getFrame(sec):
    vidcap2.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
    hasFrames,image = vidcap2.read()
    if hasFrames:
      cv2.imwrite(pathframes+"/image_%d.png"  % count , image)
      #cv2.imwrite("/content/drive/MyDrive/Thesis/Thesis_Dataset/IMages/image_%d.png", count, image)
    return hasFrames

#vidcap = cv2.VideoCapture('/content/drive/MyDrive/Thesis/Thesis_Dataset/VideosSpliting/Test/')
listarra=[]
for g in os.listdir('/content/drive/MyDrive/testing_folder/videos'):
  j=g.split(".")
  listarra.append(j[0])
print(listarra)
for f in os.listdir('/content/drive/MyDrive/testing_folder/videos'):
  foldername=f.split(".")
  vidcap2 = cv2.VideoCapture('/content/drive/MyDrive/testing_folder/videos/'+f)
  #print(vidcap2)
  if foldername[0] in listarra:
  #  print("dd")
  
    #["sjh57ujp52M", "bAC0cZIQVOk", "_H0Mmr15_zA", "5ZWUiD2OL04", "_Vzpj0cXoSM", "Afp_1HegZOc","9GIPE0aeVNI", "2NsbFst90QQ", "6c7WtnF5XWg", 
                          #   "W6DgS0s0qcI", "FfyOV00sXEY", "U_yVc8Dl048", "xx698BRyqG4"] :
    pathframes = "/content/drive/MyDrive/testing_folder/timebased2/"+foldername[0]

    # print(pathframes)
    os.mkdir(pathframes)
      
    sec = 0
    frameRate = 1 #//it will capture image in each 2 second
    count=1
    success = getFrame(sec)
    while success:
        #  if count > 120:
        #    break;
         # else:
            sec = sec + frameRate
            sec = round(sec, 2)
            print(sec)
            success = getFrame(sec)
            count = count + 1


In [ ]:
#**framesapproved


source ="/content/drive/MyDrive/testing_folder/timebased2/"
#path="/content/drive/MyDrive/Thesis/Thesis_Dataset/SSIM_Based/"
dest="/content/drive/MyDrive/testing_folder/ExtractFrame_SSIM2/"

for folder in os.listdir(source):
  #  namefile=folder
    nn=source+folder+"/"
    n2=dest+folder+"/"
    os.makedirs(n2)
    print(n2)
    array_size=0
    array_lengh=0
    image_array = [] #original Images
    count=0
    cc=0
    gray_array = [] #Gray Images

    for myFile1 in os.listdir(nn):
        array_size=array_size+1
        imageA = cv2.imread(nn+myFile1)
        image_array.append(imageA) 
        print("myFile1", myFile1)

        #print("4",imageA)
      #  cv2_imshow(imageA)

    #array_lengh=math.ceil((array_size)/2)

    #for myFile2 in os.listdir(nn):
       # os.remove(nn+myFile2)

    for img in image_array:
        grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # new_im = Image.fromarray(grayimg)
        gray_array.append(grayimg)
        #cv2_imshow(grayimg)
        print("11")


    for i in range(0, len(gray_array), 2):
        j=i+1
     
       # print(i, j, count, cc)
        if j==len(gray_array) :
          cv2.imwrite(n2+"Image_%d.png" % count,image_array[i])
          count=count+1
          print("22")


        else:
          (score, diff) = compare_ssim(gray_array[i], gray_array[j], full=True)
          if score < 0.8 :
            cv2.imwrite(n2+"Image_%d.png" % count ,image_array[i])
            count=count+1
            cv2.imwrite(n2+"Image_%d.png" % count, image_array[j])
            count=count+1
            #cv2_imshow(image_array[i])
            print("33")


            
          else:
            #print(topath, gray_array)
            cv2.imwrite(n2+"Image_%d.png" % count, image_array[i])
            count=count+1
            #cv2_imshow(image_array[i])
            print("44")

In [ ]:


d="/content/drive/MyDrive/testing_folder"


def video_to_frames(video):
  video_id = video.split(".")[0]

  image_list=[]
  for i in os.listdir("/content/drive/MyDrive/testing_folder/ExtractFrame_SSIM2/"+video_id):
    image_list.append(i)
    return image_list

def load_image(name, video_path):
  
    img = cv2.imread("/content/drive/MyDrive/testing_folder/ExtractFrame_SSIM2/"+video_path+"/"+name)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    """
    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """
    video_id = video.split(".")[0]
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i], video_id)
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    # cleanup
    #shutil.rmtree(os.path.join(config.test_path, 'temporary_images'))
    return img_feats


def extract_feats_pretrained_cnn():
    """
    saves the numpy features from all the videos
    """
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(d, 'ExtractFeatures')):
        os.mkdir(os.path.join(d, 'ExtractFeatures'))

    video_list = os.listdir(os.path.join(d, 'videos'))
    for video in video_list:

        outfile = os.path.join(d, 'ExtractFeatures', video.split(".")[0] + '.npy')
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)


if __name__ == "__main__":
    extract_feats_pretrained_cnn()

In [ ]:
def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

In [ ]:
pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
